In [1]:
!pip install lxml
!pip install et_xmlfile
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

In [3]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [25]:
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
       
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
        
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["Postalcode"] = df["Postalcode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

TypeError: sequence item 4: expected str instance, int found

In [28]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [29]:
df.head()

,Postalcode,Borough,Neighborhood
1,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge..."
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
3,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
4,M1G,Scarborough,"Woburn, Woburn, Woburn, Woburn"
5,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae, Cedarbrae"


In [33]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [34]:
print(df2.shape)
df2.head(10)

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge..."
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae, Cedarbrae"
5,M1J,Scarborough,"Scarborough Village, Scarborough Village, Scar..."
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park, K..."
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge, Golden Mile, ..."
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village Wes..."
9,M1N,Scarborough,"Birch Cliff, Cliffside West, Birch Cliff, Clif..."


In [35]:

print('The DataFrame shape is', df2.shape)

The DataFrame shape is (103, 3)
